<a href="https://colab.research.google.com/github/AreLunelle/PhytonSQL/blob/main/AreBot_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerias

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Dataset Textual

In [14]:
frases = [
    "hola", "buenos días", "buenas tardes",     # saludo
    "te amo", "me encantas", "te quiero mucho", "me haces feliz", "me gustas",    # amor
    "te odio", "vete", "no me hables", "me haces llorar", "estoy triste", "me caes mal", "lo siento"   # enojo/tristeza
]

etiquetas = [0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]


Tokenizar

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(frases)
secuencias = tokenizer.texts_to_sequences(frases)

# Emparejar el tamaño de las secuencias
padded_secuencias = pad_sequences(secuencias) #rellena con ceros para que todas tengan el mismo tamaño

Red Neuronal

In [16]:
model = Sequential([
    Embedding(input_dim=50, output_dim=8, input_length=padded_secuencias.shape[1]),
    Flatten(),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')  # 3 clases: saludo, amor, enojo
])
#Embedding convierte palabras en numeros
#Dense es el corazón neuronal
#Softmax probabilidad para cada categoria

Entrenar

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

import numpy as np

padded_secuencias = np.array(padded_secuencias)
etiquetas = np.array(etiquetas)

model.fit(padded_secuencias, etiquetas, epochs=30, verbose=1)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9333 - loss: 0.8747
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9333 - loss: 0.8699
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9333 - loss: 0.8649
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.9333 - loss: 0.8600
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9333 - loss: 0.8551
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.9333 - loss: 0.8502
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9333 - loss: 0.8453
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9333 - loss: 0.8404
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9333 - loss: 0.8354
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.9333 - loss: 0.8305
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9333 - loss: 0.8255
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9333 - loss: 0.8205
Ep

Predicciones

In [22]:
nueva_frase = ["Te quiero mucho"]
nueva_sec = tokenizer.texts_to_sequences(nueva_frase)
nueva_pad = pad_sequences(nueva_sec, maxlen=padded_secuencias.shape[1])

pred = model.predict(nueva_pad)
print("Predicción:", tf.argmax(pred, axis=1).numpy())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Predicción: [1]


Texto

In [23]:
emociones = ["saludo", "amor", "enojo/tristeza"]
print("Emoción detectada:", emociones[tf.argmax(pred, axis=1).numpy()[0]])

Emoción detectada: amor


Interacción tiempo real

In [24]:
emociones = ["saludo", "amor", "enojo/tristeza"]
respuestas = {
    "saludo": "¡Hola! Qué gusto saludarte 😊",
    "amor": "Siento amor en tus palabras. Qué bonito 💕",
    "enojo/tristeza": "Tu tono suena triste. ¿Todo bien? 😢"
}

while True:
    frase_usuario = input("Escribe una frase (o 'salir' para terminar): ")
    if frase_usuario.lower() == "salir":
        print("👋 ¡Hasta luego!")
        break

    secuencia = tokenizer.texts_to_sequences([frase_usuario])
    secuencia_padded = pad_sequences(secuencia, maxlen=padded_secuencias.shape[1])

    pred = model.predict(secuencia_padded)
    emocion_detectada = emociones[tf.argmax(pred, axis=1).numpy()[0]]

    print("🔍 Emoción detectada:", emocion_detectada)
    print("🤖 Respuesta del bot:", respuestas[emocion_detectada])
    print()


Escribe una frase (o 'salir' para terminar): ¡Hola, te extrañé!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
🔍 Emoción detectada: enojo/tristeza
🤖 Respuesta del bot: Tu tono suena triste. ¿Todo bien? 😢

Escribe una frase (o 'salir' para terminar): Si, estoy bien. Un poco triste por que queria salir a jugar. Pero no importa.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
🔍 Emoción detectada: amor
🤖 Respuesta del bot: Siento amor en tus palabras. Qué bonito 💕

Escribe una frase (o 'salir' para terminar): ¡No me hables!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🔍 Emoción detectada: enojo/tristeza
🤖 Respuesta del bot: Tu tono suena triste. ¿Todo bien? 😢

Escribe una frase (o 'salir' para terminar): No, NO QUIERO HABLAR. YA TE DIJE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
🔍 Emoción detectada: enojo/tristeza
🤖 Respuesta del bot: Tu tono suena triste. ¿Todo bien? 😢

Escribe una frase (o 'salir' para terminar): Bueno, esta bien. Te quiero mucho.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
🔍 Emoción detectada: amor
🤖 Respu